# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`type IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [ ]:
object StdDefinition {
    // type Option[T] = 1 + T
    // type Either[S, T] = S + T 
    // type List[T] = 1 + T * List[T]
    //              = 1+T*(1+T*List[T])=1+T*1+T*T*List[T]=1+T+T^2*List[T]
    //              = 1 + T + T^2 + T^3 + T^4 + ...
}

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [0]:
val s: String = 1

cmd0.sc:1: type mismatch;
 found   : Int(1)
 required: String
val s: String = 1
                ^Compilation Failed

: 

In [1]:
def foo(n: Nothing): Int = n

defined function foo

In [2]:
object ActualStdDefinition{
    sealed abstract class MiList[T]
    case class Empty[T](u: Unit) extends MiList[T]
    case class NonEmpty[T](head: T, tail: MiList[T]) extends MiList[T]
    
    sealed abstract class List[+T]
    case object Nil extends List[Nothing]
    case class ::[T](head: T, tail: List[T]) extends List[T]
    
    val l3: List[Int] = Nil : List[Nothing]
    val l2: List[Int] = ::[Int](1, 
                                ::(2, 
                                   ::[Int](3, 
                                           Nil: List[Int]
                                    )
                                )
                        )
    
    // l = [1,2,3]
    val l: MiList[Int] = 
        NonEmpty(1, NonEmpty(2, NonEmpty(3, Empty())))
}

defined object ActualStdDefinition

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [ ]:
// Less beautifully 



// More idiomatically

val l = List(1,2,3)
val l2: List[Int] =  1 :: 2 :: 3 :: Nil
val l3: List[Int] = ::(1, ::(2, ::(3, Nil)))

And can also pattern match lists, similarly:

In [5]:
// Less beautifully

// more idiomatically

val o: Option[Int] = Some(1)
o match {
    case None => "" 
    case Some(i) => "" 
}

val e: Either[Int, String] = Left(1)
e match {
    case Left(i: Int) => ""
    case Right(s: String) => ""
}

val l: List[Int] = List(1,2,3)
l match {
    case Nil => "empty" 
    case ::(head: Int, tail: List[Int]) => "nonempty"
}

val l2: List[Int] = List(1,2,3)
l match {
    case Nil => "empty" 
    case head :: tail => "nonempty"
}

// or


o: Option[Int] = Some(1)
res4_1: String = ""
e: Either[Int, String] = Left(1)
res4_3: String = ""
l: List[Int] = List(1, 2, 3)
res4_5: String = "nonempty"
l2: List[Int] = List(1, 2, 3)
res4_7: String = "nonempty"

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [16]:
// Using mutable variables

def lengthI[T](list: List[T]): Int = {
    var out: Int = 0
    for (e <- list)
        out = out + 1
    return out
}
    

defined function lengthI

In [28]:
// Using mutable variables
import scala.annotation.tailrec
def lengthTR[T](list: List[T]): Int = {
    
    @tailrec
    def aux(out: Int, l: List[T]): Int = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(out+1, tail)
        }
    
    aux(0, list)
    /*
    var out: Int = 0
    for (e <- list)
        out = out + 1
    return out
    */
}
    

import scala.annotation.tailrec

defined function lengthTR

In [17]:
length(List(1,2,3,4,5))
length(List())
length(Nil)
length(1::2::3::4::5::Nil)

res16_0: Int = 5
res16_1: Int = 0
res16_2: Int = 0
res16_3: Int = 5

The recursive function is implemented as follows: 

In [30]:
// Using recursive functions
import scala.annotation.tailrec

//@tailrec
def lengthR[T](list: List[T]): Int = 
    list match {
        case Nil => 
            0 : Int
        case head :: (tail: List[T]) => 
            val tailLength: Int = lengthR(tail) 
            tailLength + 1 : Int 
    }


import scala.annotation.tailrec

//@tailrec

defined function lengthR

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

In [20]:
val listatocha: List[Int] = List.fill(200000)(1)

listatocha: List[Int] = List(
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
...

In [21]:
lengthI(listatocha)

res20: Int = 200000

In [27]:
lengthTR(listatocha)

res26: Int = 200000

In [22]:
lengthR(listatocha)

: 

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [ ]:
// Using tail-recursive functions



We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [ ]:
// First, imperatively



In [ ]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [ ]:
// Imperatively


In [ ]:
// Tail-recursive


In [ ]:
// Plain recursive


### Unit testing with `scalatest`

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [40]:
class TestLength extends FlatSpec with Matchers{
    "length of empty list" should "work" in {
        lengthR(List()) shouldBe 1
    }
    
    "length of non-empty list" should "work" in {
        lengthR(List(1)) shouldBe 1
        lengthR(List(1,2,3,4)) shouldBe 4
    }
}

defined class TestLength

In [53]:
class TestLength(lengthF: List[Int] => Int) 
extends FlatSpec with Matchers{
    
    "length of empty list" should "work" in {
        lengthF(List()) shouldBe 0
    }
    
    "length of non-empty list" should "work" in {
        lengthF(List(1)) shouldBe 1
        lengthF(List(1,2,3,4)) shouldBe 4
    }
}

defined class TestLength

In [54]:
class TestLength(lengthF: List[Boolean] => Int) 
extends FlatSpec with Matchers{
    
    "length of empty list" should "work" in {
        lengthF(List()) shouldBe 0
    }
    
    "length of non-empty list" should "work" in {
        lengthF(List(true)) shouldBe 1
        lengthF(List(false,true,false,false)) shouldBe 4
    }
}

defined class TestLength

In [52]:
lengthR[Char](List('a', 'b', 'c'))
lengthR[Int](List(1,2,3))
lengthR[Boolean](List(true, false, true))

res51_0: Int = 3
res51_1: Int = 3
res51_2: Int = 3

In [56]:
//def lengthR[T](list: List[T]): Int = 
run(new TestLength(lengthR[Boolean]))
run(new TestLength(lengthTR[Boolean]))
run(new TestLength(lengthI[Boolean]))

cmd53$Helper$TestLength:
length of empty list
- should work
length of non-empty list
- should work
cmd53$Helper$TestLength:
length of empty list
- should work
length of non-empty list
- should work
cmd53$Helper$TestLength:
length of empty list
- should work
length of non-empty list
- should work


The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [57]:
class TestSum(sum: List[Int] => Int) 
extends FlatSpec with Matchers{
    "sum" should "work" in {
        sum(List()) shouldBe 0
        sum(List(1,2,3,4)) shouldBe 10
        sum(List(4)) shouldBe 4
    }
}

defined class TestSum

In [ ]:
// Recursively

def sum(list: List[Int]): Int = 
    list match {
        case Nil => 
            ??? : Int
        case head :: (tail: List[Int]) => 
            val tailSum: Int = ??? 
            ??? : Int 
    }

In [58]:
// Recursively

def sum(list: List[Int]): Int = 
    list match {
        case Nil => 
            0 : Int
        case head :: (tail: List[Int]) => 
            val tailSum: Int = sum(tail) 
            head + tailSum : Int 
    }

defined function sum

In [59]:
run(new TestSum(sum))

cmd56$Helper$TestSum:
sum
- should work


In [ ]:
// With tail-recursion

def sumTR(l: List[Int]): Int = {
    def aux(out: Int, l: List[Int]): Int = 
        l match {
            case Nil => ??? 
            case e :: tail => 
                aux(???, tail)
        } 
    
    aux(???, l)
}

In [60]:
// With tail-recursion

def sumTR(l: List[Int]): Int = {
    def aux(out: Int, l: List[Int]): Int = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(out + e, tail)
        } 
    
    aux(0, l)
}

defined function sumTR

### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [62]:
class TestProduct(product: List[Int] => Int) 
extends FlatSpec with Matchers{
    "product" should "work" in {
        product(List(1,2,3,4)) shouldBe 24
        product(List(1,2,3,0,6,7,8,8,10)) shouldBe 0
        product(List(1,5)) shouldBe 5
        product(List()) shouldBe 1
    }
}

defined class TestProduct

This is the common recursive implementation:

In [63]:
def multiply(l: List[Int]): Int = 
    l match {
        case Nil => ??? 
        case head :: tail => 
            val tailMult: Int = ??? 
            ??? : Int
    }

defined function multiply

In [64]:
def multiply(l: List[Int]): Int = 
    l match {
        case Nil => 1
        case head :: tail => 
            val tailMult: Int = multiply(tail)
            head * tailMult : Int
    }

defined function multiply

In [64]:
def multiply(l: List[Int]): Int = 
    l match {
        case Nil => 1
        case head :: tail => 
            if (head == 0) 0
            else {
                val tailMult: Int = multiply(tail)
                head * tailMult : Int
            }
    }

defined function multiply

In [66]:
def multiply(l: List[Int]): Int = 
    l match {
        case Nil => 1
        case head :: tail if head == 0 => 0
        case head :: tail => 
            val tailMult: Int = multiply(tail)
            head * tailMult : Int
    }

defined function multiply

In [66]:
def multiply(l: List[Int]): Int = 
    l match {
        case Nil => 1
        case 0 :: tail => 0
        case head :: tail => 
            val tailMult: Int = multiply(tail)
            head * tailMult : Int
    }

defined function multiply

It works as expected: 

In [65]:
run(new TestProduct(multiply))

cmd61$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
/*def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => head * product(tail)
    }
    */

In [ ]:
run(TestProduct)

A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [ ]:
object TestMember extends FlatSpec with Matchers{
    "member" should "work" in {
    }
}

In [ ]:
run(TestMember)

We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [2]:
class TestLast(last: List[Int] => Option[Int]) 
extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1,2)) shouldBe Some(2)
        last(List(1)) shouldBe Some(1)
        last(List()) shouldBe None
    }
}

defined class TestLast

In [3]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = last(tail)
            tailSol match {
                case None => 
                    Some(head) : Option[A]
                case Some(e) => 
                    Some(e) : Option[A]
            }
    }

defined function last

In [5]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None 
        case head :: tail => 
            last(tail) match {
                case None => 
                    Some(head) 
                case s => s
            }
    }

defined function last

In [6]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None 
        case head :: Nil => 
            Some(head)
        case _ :: tail => 
            last(tail)
    }

defined function last

In [7]:
run(new TestLast(last))

cmd1$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [ ]:
object TestInsertLast extends FlatSpec with Matchers{
    "insertLast" should "work" in {
    }
}

In [ ]:
run(TestInsertLast)

### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [12]:
List(1,2,3) :+ 4

res11: List[Int] = List(1, 2, 3, 4)

In [14]:
class TestReverse(reverse: List[Int] => List[Int]) 
extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(1 :: List(2,3,4,5)) shouldBe 
                List(5,4,3,2) :+ 1
        reverse(List(1,2)) shouldBe List(2,1)
        reverse(List()) shouldBe List()
    }
}

defined class TestReverse

In [16]:
// Recursively: Really inefficient 

def reverseR[A](l: List[A]): List[A] = 
    l match {
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailReversed: List[A] = reverseR(tail)
            tailReversed :+ head : List[A]
    }

defined function reverseR

In [17]:
run(new TestReverse(reverseR))

cmd13$Helper$TestReverse:
reverse
- should work


In [18]:
// Tail-recursive, efficiently

def reverseTR[A](list: List[A]): List[A] = {
    def aux(out: List[A], l: List[A]): List[A] = 
        l match {
            case Nil => out
            case head :: tail => 
                aux(head :: out, tail)
        }

    aux(Nil, list)
}



defined function reverseTR
res17_1: List[Int] = List(1, 2, 3, 4, 5)
res17_2: Nil.type = List()
res17_3: List[Int] = List(5, 4, 3, 2, 1)

In [20]:
// Tail-recursive, efficiently

def reverseI[A](list: List[A]): List[A] = {
    var out: List[A] = Nil
    for (head <- list)
       out = head :: out
    out
}



defined function reverseI

In [21]:
run(new TestReverse(reverseI))

cmd13$Helper$TestReverse:
reverse
- should work


In [19]:
run(new TestReverse(reverseTR))

cmd13$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)